# 과제 10-5장 Linear Regression - 자전거 통행량 예측

시애틀 프리몬트 다리를 지나는 자전거 통행량을 날씨와 계절, 기타 요인에 따라 예측하시오. (p436 참고)

<img src="https://images.seattletimes.com/wp-content/uploads/2020/01/01022020_bike-count_114545.jpg?d=2040x1360" width=400>

Ken Lambert, "Bike ridership hits record highs on 2 Seattle routes", The Seattle Times, Jan. 6, 2020

1. 교재에서 제공하는 FremontBridge.csv 파일과 SeattleWeather.csv 파일을 읽어 pandas 의 DataFrame 을 각각 생성하시오. (https://github.com/jakevdp/bicycle-data)
2. 2020 년 이전 데이터만 선택하시오.
3. 일별 총 자전거 통행량을 계산하고, 그 계산 결과를 'counts' 컬럼에 추가하시오.
4. 월요일~일요일까지 요일을 이진 데이터로 각각 인코딩 하고, 그 결과를 요일을 나타내는 컬럼들(7개)에 추가하시오.
5. 공휴일 여부를 이진 데이터로 인코딩 하고, 그 결과를 'holiday' 컬럼에 추가하시오.
6. 일자별 평균 날씨 (TAVG) 를 계산한 후, 'temp' 컬럼에 추가하시오.
7. 일자별 강수량(PRCP)을 계산한 후, 'rainfall' 컬럼에 추가하시오.
8. 데이터에서 NA 를 제거하시오.
9. LinearRegression 모델을 이용하여 자전거 통행량을 예측하시오.
10. Matplotlib 를 이용하여 예측 결과를 시각화 하시오.


In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb


In [27]:
FB_read = pd.read_csv('FremontBridge.csv', index_col='Date', parse_dates=True).sort_index()
FB_read.columns = ['Total', 'East', 'West']
SW_read = pd.read_csv('SeattleWeather.csv', index_col= 'DATE', parse_dates=True).sort_index()

print(FB_read)

C:\Users\user\AppData\Local\Temp\ipykernel_6816\170574722.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  FB_read = pd.read_csv('FremontBridge.csv', index_col='Date', parse_dates=True).sort_index()


                     Total  East  West
Date                                  
2012-10-03 00:00:00   13.0   4.0   9.0
2012-10-03 00:00:00   13.0   4.0   9.0
2012-10-03 01:00:00   10.0   4.0   6.0
2012-10-03 01:00:00   10.0   4.0   6.0
2012-10-03 02:00:00    2.0   1.0   1.0
...                    ...   ...   ...
2021-12-31 19:00:00    0.0   0.0   0.0
2021-12-31 20:00:00    0.0   0.0   0.0
2021-12-31 21:00:00    0.0   0.0   0.0
2021-12-31 22:00:00    0.0   0.0   0.0
2021-12-31 23:00:00    0.0   0.0   0.0

[147278 rows x 3 columns]


In [46]:
FB_read[: '2019']
#FB_read = FB_read[FB_read.index < "2020-01-01"]
#X = FB_read[pd.DatetimeIndex(FB_read.index).year < 2020] 

,Total,East,West
Date,,,
2012-10-03 00:00:00,13.0,4.0,9.0
2012-10-03 00:00:00,13.0,4.0,9.0
2012-10-03 01:00:00,10.0,4.0,6.0
2012-10-03 01:00:00,10.0,4.0,6.0
2012-10-03 02:00:00,2.0,1.0,1.0
...,...,...,...
2019-12-31 21:00:00,15.0,8.0,7.0
2019-12-31 22:00:00,15.0,5.0,10.0
2019-12-31 22:00:00,15.0,5.0,10.0


In [53]:
daily = FB_read.resample('d').sum()
print(daily)


             Total    East    West
Date                              
2012-10-03  7042.0  3520.0  3522.0
2012-10-04  6950.0  3416.0  3534.0
2012-10-05  6296.0  3116.0  3180.0
2012-10-06  4012.0  2160.0  1852.0
2012-10-07  4284.0  2382.0  1902.0
...            ...     ...     ...
2021-12-27    74.0    23.0    51.0
2021-12-28    91.0    32.0    59.0
2021-12-29    75.0    32.0    43.0
2021-12-30    77.0    37.0    40.0
2021-12-31    52.0    31.0    21.0

[3377 rows x 3 columns]


In [55]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)

In [56]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2020')
daily = daily.join(pd.Series(1, index=holidays, name='holidays'))

In [ ]:
SW_read['Temp (F)'] =0.5 * (SW_read['TMIN'] + SW_read['TMAX'])